In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part2/stellar_X_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part2/stellar_y_train.csv')

X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part2/stellar_X_test.csv')

<br>

주어진 학습용 데이터를 활용하여 해당 천체가 은하인지 아닌지 예측 모형을 만든 후,

이를 평가용 데이터 (stellar_X_test.csv)에 적용하여 얻은 천체의 은하 여부를 다음과 같은 형식의 csv 파일로 생성하세요.

<br>

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     70000 non-null  float64
 1   delta     70000 non-null  float64
 2   u         70000 non-null  float64
 3   g         70000 non-null  float64
 4   r         70000 non-null  float64
 5   i         70000 non-null  float64
 6   z         70000 non-null  float64
 7   redshift  70000 non-null  float64
 8   run_ID    70000 non-null  int64  
 9   rerun_ID  70000 non-null  int64  
 10  cam_col   70000 non-null  int64  
 11  field_ID  70000 non-null  int64  
 12  plate     70000 non-null  int64  
dtypes: float64(8), int64(5)
memory usage: 6.9 MB


In [6]:
'''
데이터 전처리

모두 다 수치 데이터이며, 결측치가 없다.

이상치가 있는지 확인해보도록 하자.

u, g, z 에 이상치가 있는 것으로 확인된다.

'''

X_train.describe()

,alpha,delta,u,g,r,i,z,redshift,run_ID,rerun_ID,cam_col,field_ID,plate
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.0,70000.000000,70000.000000,70000.000000
mean,177.535809,24.107885,21.938806,20.486718,19.644011,19.082969,18.624590,0.575246,4478.649443,301.0,3.510500,186.045714,5127.852900
std,96.483093,19.665057,37.942814,37.925201,1.852053,1.755997,37.904705,0.729921,1965.326847,0.0,1.585203,148.661268,2947.452991
min,0.005528,-18.785328,-9999.000000,-9999.000000,9.822070,9.469903,-9999.000000,-0.009971,109.000000,301.0,1.000000,11.000000,266.000000
25%,127.385304,5.115922,20.349987,18.957325,18.136183,17.733732,17.463838,0.052602,3185.000000,301.0,2.000000,82.000000,2525.750000
50%,180.706865,23.579507,22.183860,21.095445,20.122805,19.399590,19.001010,0.422838,4188.000000,301.0,4.000000,146.000000,4978.000000
75%,233.942154,39.871336,23.684145,22.121073,21.039012,20.391340,19.917578,0.700399,5326.000000,301.0,5.000000,241.000000,7389.000000
max,359.999615,82.947622,32.781390,31.602240,29.571860,32.141470,28.790550,7.011245,8162.000000,301.0,6.000000,989.000000,12547.000000


In [11]:
columns_outlier = ['u', 'g', 'z']

q = []
for col in columns_outlier:
    q.append(X_train[col].quantile(0.25).tolist())
q

[20.349987499999997, 18.957325, 17.4638375]

In [36]:
dataset = pd.concat([X_train, y_train], axis=1)
dataset

,alpha,delta,u,g,r,i,z,redshift,run_ID,rerun_ID,cam_col,field_ID,plate,galaxy
0,15.543143,11.633094,22.93538,20.90945,19.21472,18.55761,18.25810,0.351560,7787,301,2,303,4663,1
1,229.830659,22.738286,19.23876,18.55859,18.35505,18.30021,18.02694,2.342957,4649,301,5,297,3955,0
2,197.034525,4.473539,21.53487,20.89188,20.05817,19.31662,18.85004,0.654545,2247,301,3,242,4758,1
3,350.480205,33.899168,21.74429,21.16560,20.75535,20.17831,19.89537,1.722429,8157,301,3,66,7757,0
4,2.394550,-4.361202,22.31607,22.79926,21.83227,21.34860,21.70997,0.852792,7824,301,5,332,9344,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,49.151704,0.865266,22.07175,19.17831,17.79634,17.17832,16.82632,0.000088,3438,301,6,128,1065,0
69996,336.558707,17.420428,19.27420,17.41205,16.37569,15.86725,15.48150,0.110542,8101,301,3,78,5030,1
69997,223.505657,14.380025,19.85915,18.98436,18.64013,18.52208,18.49347,-0.000344,5326,301,5,155,5478,0
69998,352.367055,0.755039,24.72686,21.53885,20.22935,19.59513,18.95154,0.282975,4188,301,5,59,4212,1


In [37]:
dataset_drop_outlier = dataset[(dataset[columns_outlier[0]] > -9999) | (dataset[columns_outlier[1]] > -9999) | (dataset[columns_outlier[2]] > -9999)]

In [38]:
dataset_drop_outlier

,alpha,delta,u,g,r,i,z,redshift,run_ID,rerun_ID,cam_col,field_ID,plate,galaxy
0,15.543143,11.633094,22.93538,20.90945,19.21472,18.55761,18.25810,0.351560,7787,301,2,303,4663,1
1,229.830659,22.738286,19.23876,18.55859,18.35505,18.30021,18.02694,2.342957,4649,301,5,297,3955,0
2,197.034525,4.473539,21.53487,20.89188,20.05817,19.31662,18.85004,0.654545,2247,301,3,242,4758,1
3,350.480205,33.899168,21.74429,21.16560,20.75535,20.17831,19.89537,1.722429,8157,301,3,66,7757,0
4,2.394550,-4.361202,22.31607,22.79926,21.83227,21.34860,21.70997,0.852792,7824,301,5,332,9344,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,49.151704,0.865266,22.07175,19.17831,17.79634,17.17832,16.82632,0.000088,3438,301,6,128,1065,0
69996,336.558707,17.420428,19.27420,17.41205,16.37569,15.86725,15.48150,0.110542,8101,301,3,78,5030,1
69997,223.505657,14.380025,19.85915,18.98436,18.64013,18.52208,18.49347,-0.000344,5326,301,5,155,5478,0
69998,352.367055,0.755039,24.72686,21.53885,20.22935,19.59513,18.95154,0.282975,4188,301,5,59,4212,1


In [39]:
'''

rerun의 경우에는 카디널리티가 1이라 의미가 없을 거라 판단하여 제거

'''

dataset_drop_outlier['rerun_ID'].unique()

array([301])

In [48]:
dataset_drop = dataset_drop_outlier.drop(columns='rerun_ID', axis=1)

X_test = X_test.drop(columns='rerun_ID', axis=1)

In [41]:
dataset_drop

,alpha,delta,u,g,r,i,z,redshift,run_ID,cam_col,field_ID,plate,galaxy
0,15.543143,11.633094,22.93538,20.90945,19.21472,18.55761,18.25810,0.351560,7787,2,303,4663,1
1,229.830659,22.738286,19.23876,18.55859,18.35505,18.30021,18.02694,2.342957,4649,5,297,3955,0
2,197.034525,4.473539,21.53487,20.89188,20.05817,19.31662,18.85004,0.654545,2247,3,242,4758,1
3,350.480205,33.899168,21.74429,21.16560,20.75535,20.17831,19.89537,1.722429,8157,3,66,7757,0
4,2.394550,-4.361202,22.31607,22.79926,21.83227,21.34860,21.70997,0.852792,7824,5,332,9344,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,49.151704,0.865266,22.07175,19.17831,17.79634,17.17832,16.82632,0.000088,3438,6,128,1065,0
69996,336.558707,17.420428,19.27420,17.41205,16.37569,15.86725,15.48150,0.110542,8101,3,78,5030,1
69997,223.505657,14.380025,19.85915,18.98436,18.64013,18.52208,18.49347,-0.000344,5326,5,155,5478,0
69998,352.367055,0.755039,24.72686,21.53885,20.22935,19.59513,18.95154,0.282975,4188,5,59,4212,1


In [42]:
'''
이제 다른 특이한 값은 없는 것으로 판단되어 학습용 데이터셋과 검증용 데이터셋으로 분리하는 작업을 수행


'''

dataset_drop.describe()

,alpha,delta,u,g,r,i,z,redshift,run_ID,cam_col,field_ID,plate,galaxy
count,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000,69999.000000
mean,177.535145,24.108238,22.081964,20.629856,19.644032,19.082984,18.767701,0.575254,4478.702681,3.510522,186.040701,5127.878813,0.593808
std,96.483622,19.664976,2.248554,2.033576,1.852057,1.756005,1.764734,0.729923,1965.290408,1.585204,148.656411,2947.466071,0.491125
min,0.005528,-18.785328,10.996230,10.498200,9.822070,9.469903,9.612333,-0.009971,109.000000,1.000000,11.000000,266.000000,0.000000
25%,127.384192,5.117238,20.350000,18.957435,18.136075,17.733685,17.463885,0.052608,3185.000000,2.000000,82.000000,2525.500000,0.000000
50%,180.705886,23.580630,22.183900,21.095450,20.122810,19.399590,19.001020,0.422848,4188.000000,4.000000,146.000000,4978.000000,1.000000
75%,233.942330,39.872079,23.684160,22.121075,21.039035,20.391350,19.917595,0.700399,5326.000000,5.000000,241.000000,7389.000000,1.000000
max,359.999615,82.947622,32.781390,31.602240,29.571860,32.141470,28.790550,7.011245,8162.000000,6.000000,989.000000,12547.000000,1.000000


In [45]:
columns_label = ['galaxy']
columns_num = [col for col in dataset_drop.columns if col not in columns_label]

X_train = dataset_drop[columns_num]
y_train = dataset_drop[columns_label]

In [46]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

In [49]:
'''
스케일링을 진행
'''

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr)

X_tr = scaler.transform(X_tr)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

/Users/wooy0ng/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [53]:
'''
변수 중요도를 체크하기 위해 RamdomForest 모델을 초기 모델로 설정
'''

from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=500, max_depth=10, verbose=100)
model_RF.fit(X_tr, y_tr)

/var/folders/xj/8qmq115149l4lrq0qv4w4d1c0000gn/T/ipykernel_4134/3640576089.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_RF.fit(X_tr, y_tr)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 500
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
building tree 2 of 500
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
building tree 3 of 500
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
building tree 4 of 500
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
building tree 5 of 500
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s remaining:    0.0s
building tree 6 of 500
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s remaining:    0.0s
building tree 7 of 500
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.5s remaining:    0.0s
building tree 8 of 500
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.5s remaining:    0.0s
building tree 9 of 500
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.6s remaining: 

RandomForestClassifier(max_depth=10, n_estimators=500, verbose=100)

In [55]:
from sklearn.metrics import roc_auc_score

y_val_pred = model_RF.predict_proba(X_val)
print(roc_auc_score(y_val, y_val_pred[:, 1]))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [57]:
for col, importance in zip(columns_num, model_RF.feature_importances_):
    print(f"{col}: {importance}")

alpha: 0.00723026826999087
delta: 0.007119966789931181
u: 0.06796363749626884
g: 0.08513958447349654
r: 0.0386009856589937
i: 0.06114002220362245
z: 0.10495377262101337
redshift: 0.5642501155073891
run_ID: 0.005531311892818277
cam_col: 0.001152936224732336
field_ID: 0.003124906759394015
plate: 0.05379249210234927
